# 필요한 라이브러리 import

In [80]:
import copy

#Javascript parser
from pyjsparser import parse

#Requets 및 html파싱 라이브러리, 크롤링 관련
import requests
from bs4 import BeautifulSoup
from selenium import  webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import base64
import re

#데이터프레임 라이브러리
import pandas as pd

#메일 전송 라이브러리
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText 
from email.mime.image import MIMEImage

#날짜 관련 라이브러리 및 변수
import datetime
now = datetime.datetime.now()
today = now.strftime('%Y%m%d')

# _____(메일 전송)

In [33]:
def email_send(total_data, cart_data, user_email):
    recipients = [user_email] # 수신인 입력
    strFrom = 'asia.ec.nlp@gmail.com'
    strTo = ", ".join(recipients)

    msgRoot = MIMEMultipart('related') #그대로 작성
    msgRoot['Subject'] = str(today) + '장바구니 주문내역'

    msgRoot['From'] = strFrom
    msgRoot['To'] = strTo
    msgRoot.preamble = 'This is a multi-part message in MIME format.'
    msgAlternative = MIMEMultipart('alternative') 
    msgRoot.attach(msgAlternative) 
    msgText = MIMEText('This is the alternative plain text message.')
    msgAlternative.attach(msgText)
    
    msgText = MIMEText("""
    <html>
    <style>
      table {
        width: 100%;
        text-align: center;
        line-height: 1.0;
        margin: 1px 1px;
      }
    </style>
    <body>
          <h3> """+ today +""" 주문 정보 </h3>
          """+total_data.to_html(index=False,justify='center',col_space=100)+"""
    </body>
    </html>
    """+cart_data.to_html(index=False,justify='center',col_space=100), 'html')
    msgAlternative.attach(msgText)
    
    s = smtplib.SMTP('smtp.gmail.com: 587')
    s.starttls()
    s.login('asia.ec.nlp@gmail.com', 'asia258258@')
    s.sendmail(strFrom, strTo, msgRoot.as_string())

    s.quit()

# ___________________________________(SSG)

In [91]:
def ssg_shopping_list(word):
    
    base_url = 'http://earlymorning.ssg.com/search.ssg'
    
    params = {'target':'all','query':word}
    resp = requests.get(base_url, params=params)
    soup = BeautifulSoup(resp.text,'lxml')


    ul_tag = soup.find('ul', class_="cunit_thmb_lst")
    items = ul_tag.find_all('li', class_='cunit_t232')

    no_item = {
        '상품명':word,
        '원가격':"0",
        '할인율':"0",
        '최종가격':"0",
        '비고':'해당상품이 없습니다.'}
    
    
    if items != []:
        item_list = []   
        for item in items[:6]:

            item_info = item.find('div',class_="cunit_info")

            #이름        
            if item_info.find('strong', class_='brd') == None:
                item_name = item_info.find('em', class_='tx_ko').text
            else:
                item_name = item_info.find_all('em', class_='tx_ko')[1].text
                    
                    
            if item.find('div',class_="soldout_area") == None:
                #할인가격
                item_price = item_info.find('em', class_='ssg_price').text

                #원가격
                item_org_price = item_info.find('div',class_ = 'org_price')
                if item_org_price ==  None:
                    item_org_price = item_price
                else:
                    item_org_price = item_info.find('div',class_ = 'org_price').find('em', class_='ssg_price').text

                #할인율(에누리, 쿠폰)
                item_sale_percent = item.find('span', class_='di_enuri')
                if item_sale_percent == None:

                    item_sale_percent = item.find('span', class_='di_coupon')
                    if item_sale_percent == None :
                        item_sale_percent = " "
                    else:
                        item_sale_percent = item.find('span', class_='di_coupon').text

                else:
                    item_sale_percent = item.find('span', class_='di_enuri').text



                #비고
                item_plus = item.find('span', class_='di_plus')
                if item_plus == None:
                    item_plus = " "
                else:
                    item_plus = item.find('span', class_='di_plus').text


                #item_list에 넣기
                item_list.append({
                    '상품명':item_name,
                    '원가격':item_org_price,
                    '할인율':item_sale_percent,
                    '최종가격':item_price,
                    '비고':item_plus
                 })
            else:
                    
                soldout_item = {
                    '상품명':item_name,
                    '원가격':"0",
                    '할인율':"0",
                    '최종가격':"0",
                    '비고':'해당상품은 품절입니다.'                   
                }
                item_list.append(soldout_item)
        
      
            
        best_item = item_list[0]
        
        
    else:
        best_item = no_item
    
        

    return best_item

In [127]:
#상품리스트 web_cart 담기
def web_cart(df_):
    data = df_.copy()
    
    cart_list = data[data['비고'] != '해당상품이 없습니다.']["상품명"].tolist()
    for word in cart_list:
    
        cart_url = 'http://earlymorning.ssg.com/search.ssg?target=all&query='

        driver.get(cart_url+word)


        thmb = driver.find_element_by_class_name('thmb')
        util_bx = driver.find_elements_by_class_name('util_bx')
        cart = util_bx[0].find_elements_by_tag_name('a')[2]

            # action
        actions = ActionChains(driver)

        actions.move_to_element(thmb)
        actions.perform()
        actions.reset_actions()

        cart = util_bx[0].find_elements_by_tag_name('a')[2]
        actions.click(cart)
        actions.perform()

        Alert(driver).accept()
        actions.reset_actions()

    #---실행방법---#
#     driver = webdriver.Chrome()
#     web_cart(df_)
#     util_menu = driver.find_element_by_class_name('util_menu')
#     final_cart = util_menu.find_elements_by_tag_name('a')[1]
#     final_cart.click()

In [3]:
# 데이터프레임 + 엑셀저장
def shoppinglist_ssg(thelist):
    data_ssg = pd.DataFrame(thelist)
    
    data_ssg["원가격"] = data_ssg["원가격"].str.replace(",",'').astype(int)
    data_ssg["최종가격"] = data_ssg["최종가격"].str.replace(",",'').astype(int)
    
    org_total = data_ssg["원가격"].sum()
    total = data_ssg["최종가격"].sum()
    
    data_ssg = data_ssg.append(
        {'상품명':"**합계**", "원가격":org_total, '할인율':total - org_total, '최종가격':total, '비고':""},
            ignore_index =  True)
    data_ssg.to_excel("data_ssg.xlsx", encoding ="cp949")
    
    return data_ssg

In [5]:
#실행코드
order_list = []
order_items = input("상품명: ")
order_list.append(order_items)
while order_items != "":
    order_items = input("")
    order_list.append(order_items)

ssglist_fin = ssg_shopping_list(order_list)
shoppinglist_ssg(ssglist_fin)

상품명: 사과
양파
당근



,상품명,원가격,할인율,최종가격,비고
0,SSG.Fresh [프리미엄] 고당도사과 1.2kg 내외 (봉),13800,,13800,
1,SSG.Fresh 수제망 햇양파 1.8Kg내외 (망),3280,,3280,
2,SSG.Fresh 햇 당근 1입 (봉),1980,,1980,
3,**합계**,19060,0,19060,


# ___________________________________(Marketkurly)

In [6]:
#패키지 상품의 코드 반환 함수
def get_package_code(item_code):
    url = 'https://api.kurly.com/v3/home/products/'
    item_code = str(item_code) + '?ver=1'
    resp = requests.get(url + item_code)
    item_pack_code = resp.json()
    package_code = 0
    
    if item_pack_code['data']['package_products'] == []:
        return 4444
    
    for i in item_pack_code['data']['package_products']:
        
        if i['is_sold_out'] == True:
            continue
        else :
            package_code = i['no']
            break
        
    return package_code

In [11]:
def items_check(product_name, keyword):
    check_keyword = keyword.split(' ')
    user_check = 0
    b_num = 1
    for check in check_keyword:
        if not check in product_name:
            if user_check >= 1:
                b_num = 1
            else: 
                b_num = 0
        user_check += 1
    return b_num

def market_kurly_cart(keyword):
    
    url_base2 = 'https://api.kurly.com/v1/search?keyword='+keyword+'&sort_type=-1&page_limit=99&page_no=1&delivery_type=0'
    
    resp = requests.get(url_base2)
    data = resp.json()
    products_list = data['data']['products']
    find_list = {}
    find_count = 0
    
    for idx in range(len(products_list)):
        if find_count > 6:
            break
        
        if products_list[idx]['sold_out'] == True:
            find_count += 1
            continue
        
        if items_check(products_list[idx]['name'], keyword) < 1:
            continue
              
        else :
            find_list = {
                    '상품명':products_list[idx]['name'],
                    '원가격':products_list[idx]['original_price'],
                    '할인율':products_list[idx]['discount_amount'],
                    '최종가격':products_list[idx]['price'],
                    '비고':products_list[idx]['shortdesc']
                }
            break
        
    if find_list.get('상품명') == None:
        find_list = {
                    '상품명':keyword,
                    '원가격':0,
                    '할인율':0,
                    '최종가격':0,
                    '비고':"상품을 찾을 수 없음"
                }
        return find_list
    
    item_code = products_list[idx]['no']
    package_code = get_package_code(item_code)
    
    if package_code == 0:
        package_code = None
    elif package_code == 4444:
        package_code = item_code
    
    kurly_item_code_list.append(package_code)
    
    return find_list

In [12]:
def getin_cart(kurly_item_code_list):
    s2 = requests.Session()
    resp = s2.get('https://www.kurly.com/shop/goods/goods_search.php?searched=Y&log=1&skey=all&hid_pr_text=%BF%EC%B8%AE%B0%A1%C1%B7+%B0%C7%B0%AD%C7%D1+%C0%E5%BA%B8%B1%E2&hid_link_url=https%3A%2F%2Fwww.kurly.com%2Fshop%2Fgoods%2Fgoods_list.php%3Fcategory%3D268&edit=Y&sword=%B9%B0&x=0&y=0')
    soup = BeautifulSoup(resp.text)
    js  = soup.find_all('script')[1].string
    a = parse(js)
    
    token = a['body'][2]['declarations'][0]['init']['value']
    
    for item_code in kurly_item_code_list:
        data = {"no":item_code,"ea":1}
        headers = {
            'authorization':token,
            'referer':'https://api.kurly.com/xdomain?ver=1',
            'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'
        }
        s2.post('https://api.kurly.com/v2/carts?ver=1',headers=headers, data=data)
    

In [ ]:
getin_cart(kurly_item_code_list)

In [17]:
#데이터 프레임
kurly_data = pd.DataFrame(cart_list)
kurly_data

,상품명,원가격,할인율,최종가격,비고
0,[KF365] 양파 1.8kg,3280,0,3280,"믿고 먹을 수 있는 상품을 합리적인 가격에, KF365"
1,GAP 아오리 사과 한봉지 1kg,6900,0,6900,여름에만 만나는 연둣빛 사과
2,[네떼]베이비 당근 250g,3600,0,3600,아담한 사이즈에 하나씩 쓰기 좋은 앙증맞은 당근(1팩/250g)
3,[비비고] 김치 볶음,2500,0,2500,100℃ 직화 솥에서 달달 볶아낸 볶음 김치!
4,[오뚜기] 채황 라면 4입,3980,0,3980,비건인증을 받은 채황 라면 4입 (110g x 4EA)
5,[농심] 신라면 건면 5입,3980,0,3980,가볍게 만나는 얼큰함의 정석(97gx5입)
6,[해올림] 무항생제 삼겹살 2종 (냉장),15900,0,15900,"100g당 판매가 3,180원"


# _______(Coupang)

# 쿠키를 사용하여 문자인증이나 로그인 없이 장바구니를 유지합니다.

### 쿠키를 이용해 장바구니를 유지하고 리스트의 상품들을 장바구니에 추가
- 같은 버전의 Chrome 과 chromedriver 가 필요합니다.
- chromedriver.exe 가 현재 파일과 같은 위치에 존재해야 합니다.

In [149]:
def add_to_cart_coupang(items) :
    # chromedriver.exe 파일 위치에 따라 변경해주어야 합니다.
    driver = webdriver.Chrome(u'chromedriver')
    driver.get("https://www.coupang.com/")
    
    # 저장된 sid를 불러와 변경함.
    try :
        with open("cookie_sid", 'r') as f :
            cookie_sid = f.read()
            cookies = driver.get_cookies()
            for cookie in cookies :
                if cookie['name'] == 'sid' :
                    cookie['value'] = cookie_sid
            for cookie in cookies:
                driver.add_cookie(cookie)
    except FileNotFoundError :
        pass
    except IOError as error:
        print("Read IOError: cookie_sid")

    for item in items :
        add_one_item(driver, item)

    # sid가 바뀌었을 수도 있으니 새로 저장.
    try :
        with open("cookie_sid", 'w+') as f :
            for cookie in driver.get_cookies() :
                if cookie['name'] == 'sid' :
                    f.write(cookie['value'])
    except IOError as error:
        print("Write IOError: cookie_sid")

In [150]:
def add_one_item(driver, item) :
    # 링크의 상품정보 페이지로 이동
    driver.get(item['링크'])
    # 상품정보 페이지 로딩 완료될때까지 대기
    element = WebDriverWait(driver, 5).until(
        EC.text_to_be_present_in_element(
            (By.NAME, 'detail'),
            '상품상세'
        )
    )
    
    # 링크의 상품 장바구니 담기
    xpath = '//button[@class="prod-cart-btn"]'
    driver.find_element_by_xpath(xpath).click()
    # 장바구니에 담길때까지 대기
    element = WebDriverWait(driver, 5).until(
        EC.text_to_be_present_in_element(
            (By.CLASS_NAME, 'prod-order-notifier-content'),
            '상품이 장바구니에 담겼습니다.'
        )
    )
    
    # 장바구니로 이동
    xpath = '//a[@class="clearFix mycart-preview-module"]'
    driver.find_element_by_xpath(xpath).click()
    # 장바구니 페이지 로딩 완료될때까지 대기
    element = WebDriverWait(driver, 5).until(
        EC.text_to_be_present_in_element(
            (By.ID, 'th-product-box'),
            '상품정보'
        )
    )

    if driver.find_element_by_xpath("//a[contains(text(), '" + item['상품명'] + "')]") is not None :
        print(item['상품명'] + ' 이 장바구니에 담겼습니다.')
    else :
        print(item['상품명'] + ' 이 장바구니에 담기지 않았습니다.')

In [151]:
# pname = 상품명
def get_cheapest_item_coupang(pname) :
    base_url = 'https://www.coupang.com/np/search'

    params = {
        'q': pname,
        'channel': 'user',
        'component': None
    }

    # 미포함시 접근 불가
    headers = {
        'Referer': 'https://www.coupang.com/',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4220.0 Safari/537.36'
    }
    
    resp = requests.get(base_url, params=params, headers=headers)
    soup = bs(resp.text)
    items = soup.select('li.search-product') # 상품정보 1개를 li.search-product 가 감싸고 있음.
    
    if soup.select('p.no-hit') : # 검색된 상품이 없을시
        return dict( # 임시 상품 반환.
            상품명 = pname,
            원가 = 0,
            할인율 = '0%',
            판매가 = 0,
            비고 = '(상품이 존재하지 않습니다)',
            링크 = None
        )
    
    # 두번째 상품부터 로켓프레시가 아니고, 검색 키워드를 포함하는 첫 상품을 검색.
    # 로켓프레시는 장바구니 담기가 안되어 제외시켜야 함.
    idxFirstItem = 1 # 첫번째 상품은 보통 광고인데 결과가 정확하지 않은 경우가 많아 제외 (사과 -> 사과식초, 배추 -> 배추김치)
    num_items = len(items)
    while idxFirstItem < num_items :
        tmpItem = items[idxFirstItem].select_one('span.badge.rocket > img') # 로켓프레시, 로켓배송 등이 가지고 있는 태그
        if pname in items[idxFirstItem].find('div', class_='name').text : # 검색 키워드를 포함하고
            if tmpItem is None : # 일반 상품(일반 상품은 장바구니 담기 가능)이면 사용.
                break
            else : # 일반 상품이 아니면(로켓프레시나 로켓배송 등)
                if 'rocket-fresh' not in tmpItem['src'] :
                    break # 로켓프레시가 아니면 사용
                else :
                    pass # 로켓프레시면 다음 상품 확인.
        idxFirstItem += 1
    
    item = items[idxFirstItem]
    info = {}
    
    # 상품명
    info['상품명'] = item.find('div', class_='name').text
     
    # 할인율, 원가, 판매가
    # span.instant-discount-rate 가 존재할 경우, 안에 할인율이 들어있음.
    objDiscountRate = item.find('span', class_='instant-discount-rate')
    price = int(re.sub("\D", "", item.find('strong', class_='price-value').text)) # 판매가
    if objDiscountRate is not None :
        info['할인율'] = int(objDiscountRate.text.replace('%',''))
        info['원가'] = int(re.sub("\D", "", item.find('del', class_='base-price').text))
        info['판매가'] = price
    else : # 할인율 정보 없을시, 0% (원가 = 판매가)
        info['할인율'] = '0%'
        info['원가'] = info['판매가'] = price
    
    # 상세정보
    # span.unit-price 가 존재할 경우, (100g당 400원) 형식으로 들어있음.
    objSpec = item.find('span', class_='unit-price')
    if objSpec is not None :
        info['비고'] = objSpec.text.strip()
    else : # 상세 정보 없을시 배송 정보로 대체
        info['비고'] = item.find('span', class_='arrival-info').text.strip()
    
    # 링크
    info['링크'] = 'https://www.coupang.com' + item.find('a', class_='search-product-link')['href']
    
    return info

In [51]:
import re
import requests
from bs4 import BeautifulSoup as bs

In [79]:
print(get_cheapest_item_coupang('대파'))
print()
print(get_cheapest_item_coupang('고추장'))
print()
print(get_cheapest_item_coupang('마늘'))
print()
print(get_cheapest_item_coupang('라면'))
print()
print(get_cheapest_item_coupang('생닭'))
print()
print(get_cheapest_item_coupang('없는상품이니까검색하지마세요'))

{'상품명': '빛고을장터 국내산 무농약 대파 3kg 1kg내외, 1봉, 01_무농약 대파 (특/1kg)', '할인율': 14, '원가': 3500, '판매가': 3000, '비고': '모레(토) 8/8   도착 예정', '링크': 'https://www.coupang.com/vp/products/1522323055?itemId=2612173783&vendorItemId=3628600382'}

{'상품명': '청정원순창 100% 현미 태양초 찰고추장, 2kg, 1개', '할인율': 56, '원가': 24600, '판매가': 10770, '비고': '(100g당 539원)', '링크': 'https://www.coupang.com/vp/products/96414?itemId=1015299&vendorItemId=3001106979'}

{'상품명': '청풍명가 2020년 명품 햇 다진마늘, 1개, 500g', '할인율': 28, '원가': 5900, '판매가': 4200, '비고': '모레(토) 8/8   도착 예정', '링크': 'https://www.coupang.com/vp/products/1309129255?itemId=2325566891&vendorItemId=70322210043&sourceType=srp_product_ads'}

{'상품명': '농심 신라면 120g x 5p + 안성탕면 125g x 5p + 얼큰 너구리 120g x 5p + 짜파게티 140g x 5p, 1세트', '할인율': '0%', '원가': 14200, '판매가': 14200, '비고': '내일(금) 8/7   도착 보장', '링크': 'https://www.coupang.com/vp/products/1910849030?itemId=3244307968&vendorItemId=70916247951'}

{'상품명': '가나안식품 생닭 대 2마리, 1세트, 2.4kg', '할인율': '0%', '원가': 9990, '판매가': 9990, '비고': '모레(토) 8/8   도착 

### 쿠팡에서 최저가 검색후 {상품명, 원가, 할인율, 판매가, 비고, 링크} 를 딕셔너리로 반환

In [152]:
def coupang_cart_add(items):
    items_list_coupang = []
    for item_dict in items:
        
        item = item_dict['상품명']

        if item_dict['링크'] is not None : items_list_coupang.append(copy.deepcopy(item))

    add_to_cart_coupang(items)

In [63]:
#종합 데이터 프레임
total_report = {
    '총 상품 수':3,
    '못 찾은 상품 수':2,
    '총 할인 금액':300,
    '총 금액':56500
}
total_report = pd.DataFrame([total_report])

# __(GS Fresh)

In [116]:
def find_price(food):
    base_url = 'https://www.gsshop.com/shop/mart/martSearch.gs'
    params={
        'supCd':'1036018',
        'mq':food,
        'lesq':'417375'
    }
    headers={
         'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'
    }

    resp = requests.get(base_url,params=params,headers=headers)
    soup = BeautifulSoup(resp.text)
    li_tags=soup.select('section#searchPrdList>ul>li')
    list1 = []
    for li in li_tags[:6]:
        #비고찾기
        img_tag =li.find('span',class_='badge-mart-label')
        if img_tag == None:
            etc=''
        else : 
            etc=img_tag.find('img').get('alt')
        
        prd_name = li.find('dt').text.strip()
        discount = li.find('span',class_='price-discount')
        prd_price = li.find('span',class_='set-price').text
        upper = li.find('del',class_='price-upper')
        link = li.find('a',class_='prd-item').get('href')
        if discount== None:
            discount = 0
            upper = 0
        else :
            discount = discount.text
            upper = int(upper.text.replace(',','')[:-1])
            
        list1.append({
            '상품명':prd_name,
            '할인율':discount,
            '원가격':upper,
            '최종가격':int(prd_price.replace(',','')[:-1]),
            '비고':etc,
            '링크':link
        })
    return list1

In [117]:
def get_list(buys):
    bucket_list=[]
    #aa=[]
    for buy in buys:
        foodlist = find_price(buy)
        #aa=sorted(flist, key=(lambda x: int(x['판매가'].replace(',','')[:-1])))
        choice={}
        name_sp = buy.split(" ")
        for f in foodlist:
            prd_name = f['상품명']
            buy_cnt=0
            for n in name_sp:
                if n in prd_name :
                    buy_cnt+=1
            if buy_cnt==len(name_sp):
                choice = f
                break;
        if choice == {} :
            choice={
                '상품명':buy,
                '할인율':0,
                '원가격':0,
                '최종가격':0,
                '비고':'제품을 찾을 수 없습니다',
                '링크':None
            }
        price= choice.get('판매가')
        bucket_list.append(choice)
    return bucket_list

In [66]:
def input_prod():
    prod_list=[]
    prod=input('상품을 입력해주세요(빈칸을 입력할경우 종료됩니다):')
    while prod!='' :
        prod_list.append(prod)
        prod=input('상품을 입력해주세요(빈칸을 입력할경우 종료됩니다):')
    return get_list(prod_list)

In [67]:
li1=input_prod()

상품을 입력해주세요(빈칸을 입력할경우 종료됩니다):사과
상품을 입력해주세요(빈칸을 입력할경우 종료됩니다):당근
상품을 입력해주세요(빈칸을 입력할경우 종료됩니다):양파
상품을 입력해주세요(빈칸을 입력할경우 종료됩니다):


In [68]:
li_df=pd.DataFrame(li1)
df = li_df.drop(['링크'],axis=1 )

In [69]:

print(li_df)
print(df)

                              상품명 할인율 원가격  최종가격    비고  \
0         [다논]액티비아(사과) 130ml x 4입          4800  행사상품   
1               GSfresh)당근 1KG(봉)          4480  전단상품   
2  [GSfresh]국산 햇양파 4~8입(1.5kg내외 망          3980  전단상품   

                                              링크  
0  /prd/prd.gs?prdid=34590443&rank=1&lseq=417383  
1  /prd/prd.gs?prdid=56008581&rank=1&lseq=417383  
2  /prd/prd.gs?prdid=59693343&rank=1&lseq=417383  
                              상품명 할인율 원가격  최종가격    비고
0         [다논]액티비아(사과) 130ml x 4입          4800  행사상품
1               GSfresh)당근 1KG(봉)          4480  전단상품
2  [GSfresh]국산 햇양파 4~8입(1.5kg내외 망          3980  전단상품


In [72]:
def select_gs(li1):
    from selenium import webdriver
    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.implicitly_wait(30) # seconds 묵시적 기다리는 것 
    
    #로그인
    driver.get('https://www.gsshop.com/cust/login/login.gs')
    my_id='sjszzang0929@naver.com'
    my_pass='gslogin2020'

    cum_id=driver.find_element_by_id('id')
    cum_id.click()
    cum_id.send_keys(my_id)
    cum_pass=driver.find_element_by_id('passwd')
    cum_pass.click()
    cum_pass.send_keys(my_pass)
    cum_login=driver.find_element_by_id('btnLogin')
    cum_login.click()
    
    #장바구니 담기
    for i in li1:
        base_url='http://www.gsshop.com'
        link = i['링크']
        if link==None:
            continue
        url=base_url+link
        driver.get(url)
        addcart =driver.find_element_by_id('addCartButton')
        addcart.click()

    #장바구니 창이동
    gocart=driver.find_element_by_id('goAddCartButtn')
    gocart.click()

In [74]:
select_gs(li1)

In [75]:
driver.close()

NameError: name 'driver' is not defined

# ------

In [34]:
email_send(total_report, kurly_data, 'karunogi@gmail.com')

# ______@@@@@(상품입력)@@@@

In [144]:
# 영수증
def total_receipt(df):
    total_count = df['상품명'].count()
    noitem = df[df['최종가격'] == 0]['상품명'].count()
    pay = df["최종가격"].sum()
    discount = pay - df["원가격"].sum()
    
    total = [{
        '총상품수':total_count,
        '못찾은상품수':noitem,
        '총할인':discount,
        '총금액':pay
    }]
    total_df = pd.DataFrame(total)
    
    return total_df

In [101]:
def ssg_df(dict_):
    data_ssg = pd.DataFrame(dict_)
    
    data_ssg["원가격"] = data_ssg["원가격"].str.replace(",",'').astype(int)
    data_ssg["최종가격"] = data_ssg["최종가격"].str.replace(",",'').astype(int)

    return data_ssg

In [153]:
######마켓컬리 변수들########
kurly_item_code_list=[]
kurly_cart_list = []
kurly_total = []
#############################
######쿠팡 변수들########
coupang_cart_add_list = []
coupang_cart_list = []
coupang_cart_total = []
#############################
######SSG 변수들########
ssg_cart_list = []
ssg_cart_total = []
#############################
######GS 변수들########
gs_key_list = []
gs_cart_list = []
gs_cart_total = []
#############################

#############사용자에게 상품 입력 받는란######################
user_keyword = input('상품이름 :')
while user_keyword != '':
    
    # 쿠팡 크롤링
    only_coupang = get_cheapest_item_coupang(user_keyword)
    coupang_cart_add_list.append(only_coupang)
    coupang_cart_list.append({
        '상품명':only_coupang['상품명'],
        '원가격':only_coupang['원가'],
        '할인율':only_coupang['할인율'],
        '최종가격':only_coupang['판매가'],
        '비고':only_coupang['비고']
    })
    
    # 마켓컬리 크롤링
    kurly_cart_list.append(market_kurly_cart(user_keyword))
    
    # GS FRESH 크롤링
    gs_key_list.append(user_keyword)
    
    # SSG 크롤링
    ssg_cart_list.append(ssg_shopping_list(user_keyword))
    
    
    user_keyword = input('상품이름 :')

gs_cart_list = get_list(gs_key_list)

##############################################################

print("마켓컬리")
kurly_data = pd.DataFrame(kurly_cart_list)
kurly_total = total_receipt(kurly_data)
print(kurly_total)

print("SSG")
ssg_data = ssg_df(ssg_cart_list)
ssg_total = total_receipt(ssg_data)
print(ssg_total)

print("쿠팡")
coupang_data = pd.DataFrame(coupang_cart_list)
coupang_total = total_receipt(coupang_data)
print(coupang_total)

print("GS")
gs_data = pd.DataFrame(gs_cart_list)
gs_total = total_receipt(gs_data)
print(gs_total)

user_email = input('이메일을 입력해주세요')
while not '@' in user_email:
    user_email = input('올바른 이메일을 입력해주세요')
 
user_select = input('구매를 진행할 마켓을 선택해주세요 (1)마켓컬리 (2)쿠팡 (3)SSG (4)GS')

while True:
    if user_select == '1':
 
        getin_cart(kurly_item_code_list)
        email_send(kurly_total,kurly_data,user_email)
        break
 
    elif user_select == '2':
 
        coupang_cart_add(coupang_cart_add_list)
        email_send(coupang_total,coupang_data,user_email)
        break
 
    elif user_select == '3':
        driver = webdriver.Chrome()
        web_cart(ssg_data)
        util_menu = driver.find_element_by_class_name('util_menu')
        final_cart = util_menu.find_elements_by_tag_name('a')[1]
        final_cart.click()
        email_send(ssg_total,ssg_data,user_email)
        break
 
    elif user_select == '4':
        select_gs(gs_cart_list)
        email_send(gs_total,gs_data,user_email)
        break
 
    elif user_select == '':
        break
 
    else:
        user_select = int(input('올바른 값을 입력해주세요(선택하지 않으려면 엔터를 누르세요)'))
        continue


상품이름 :포도
상품이름 :대파
상품이름 :다진마늘
상품이름 :
마켓컬리
   총상품수  못찾은상품수  총할인    총금액
0     3       0    0  33380
SSG
   총상품수  못찾은상품수  총할인    총금액
0     3       0    0  19280
쿠팡
   총상품수  못찾은상품수   총할인    총금액
0     3       0 -6500  80800
GS
   총상품수  못찾은상품수   총할인    총금액
0     3       0 -2680  18260


In [154]:

user_email = input('이메일을 입력해주세요')
while not '@' in user_email:
    user_email = input('올바른 이메일을 입력해주세요')
 
user_select = input('구매를 진행할 마켓을 선택해주세요 (1)마켓컬리 (2)쿠팡 (3)SSG (4)GS')
while True:
    if user_select == '1':
 
        getin_cart(kurly_item_code_list)
        email_send(kurly_total,kurly_data,user_email)
        break
 
    elif user_select == '2':
 
        coupang_cart_add(coupang_cart_add_list)
        email_send(coupang_total,coupang_data,user_email)
        break
 
    elif user_select == '3':
        driver = webdriver.Chrome()
        web_cart(ssg_data)
        util_menu = driver.find_element_by_class_name('util_menu')
        final_cart = util_menu.find_elements_by_tag_name('a')[1]
        final_cart.click()
        email_send(ssg_total,ssg_data,user_email)
        break
 
    elif user_select == '4':
        select_gs(gs_cart_list)
        email_send(gs_total,gs_data,user_email)
        break
 
    elif user_select == '':
        break
 
    else:
        user_select = int(input('올바른 값을 입력해주세요(선택하지 않으려면 엔터를 누르세요)'))
        continue

이메일을 입력해주세요karunogi@gmail.com
구매를 진행할 마켓을 선택해주세요 (1)마켓컬리 (2)쿠팡 (3)SSG (4)GS2
참조은 망고포도 샤인머스켓 거봉 켐벨포도 2kg 4kg, 망고포도 샤인머스켓 2kg (3-4송이 내외) 이 장바구니에 담겼습니다.
빛고을장터 국내산 무농약 대파 3kg 1kg내외, 1봉, 01_무농약 대파 (특/1kg) 이 장바구니에 담겼습니다.
초록들 국내산 간마늘 2020년 햇마늘 다진마늘 (100%국산마늘), 1kg, 1개 이 장바구니에 담겼습니다.
